In [ ]:
# SETUP_ARTIFACT_PATHS
from pathlib import Path
BASE_DIR = Path.cwd().resolve()
if BASE_DIR.name == 'notebooks':
    BASE_DIR = BASE_DIR.parent
MODELS_DIR = BASE_DIR / 'models'
MAPPINGS_DIR = BASE_DIR / 'mappings'
MODELS_DIR.mkdir(parents=True, exist_ok=True)
MAPPINGS_DIR.mkdir(parents=True, exist_ok=True)
MODELO_OUT = MODELS_DIR / 'modelo_char_rnn.keras'
MAPEAMENTOS_OUT = MAPPINGS_DIR / 'mapeamentos.pkl'
print('Modelo:', MODELO_OUT)
print('Mapeamentos:', MAPEAMENTOS_OUT)

## Como salvar artefatos (guia rápido)
Use os caminhos padronizados definidos na célula de setup:
- Modelo: `MODELO_OUT` (Path)
- Mapeamentos: `MAPEAMENTOS_OUT` (Path)

Recomendação prática:
- Defina `NOME_ARQUIVO_MODELO = str(MODELO_OUT)` e `NOME_ARQUIVO_MAPS = str(MAPEAMENTOS_OUT)`
- Use essas variáveis nas chamadas de `model.save(...)` e ao salvar/abrir mapeamentos (`open(NOME_ARQUIVO_MAPS, 'wb')`).

In [ ]:
import numpy as np
import tensorflow as tf
import os
import pickle
import sys

# --- 1. CONFIGURAÇÕES E CONSTANTES ---
# Usaremos "Dom Casmurro" como nosso corpus de treinamento
URL_LIVRO = "https://www.gutenberg.org/files/55752/55752-0.txt"
NOME_ARQUIVO_MODELO = str(MODELO_OUT)
NOME_ARQUIVO_MAPS = str(MAPEAMENTOS_OUT)
TAMANHO_SEQUENCIA = 160 # O modelo verá 160 caracteres para prever o 101º
EPOCAS_TREINO = 40 # Aumente para um modelo melhor, mas o treino será mais longo
BATCH_SIZE = 64
from tensorflow.keras import optimizers, callbacks
optimizer = optimizers.Adam(learning_rate=2e-3, clipnorm=1.0)


c:\Users\Cristovam\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Cristovam\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
c:\Users\Cristovam\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/

In [ ]:
# Treinamento com callbacks recomendados (pule se já possui modelo salvo)
cb = [
    callbacks.ModelCheckpoint(NOME_ARQUIVO_MODELO, save_best_only=True, monitor='loss', mode='min'),
    callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=5e-5),
    callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True),
]


In [2]:
# --- 2. FUNÇÃO DE TREINAMENTO (SÓ EXECUTA SE NECESSÁRIO) ---
def treinar_modelo():
    """
    Baixa os dados, processa, constrói e treina o modelo.
    Salva o modelo e os mapeamentos no final.
    """
    print(">>> Arquivo de modelo não encontrado. Iniciando treinamento...")
    print(">>> Este processo pode ser BEM demorado e só acontece uma vez.")

    # Baixar os dados
    caminho_arquivo = tf.keras.utils.get_file(
        "dom_casmurro.txt", URL_LIVRO
    )
    
    # Ler e processar o texto
    texto = open(caminho_arquivo, 'rb').read().decode(encoding='utf-8').lower()
    print(f">>> Corpus carregado: {len(texto)} caracteres.")

    # Criar mapeamentos
    caracteres = sorted(list(set(texto)))
    char_para_int = {c: i for i, c in enumerate(caracteres)}
    int_para_char = {i: c for i, c in enumerate(caracteres)}
    n_vocabulario = len(caracteres)

    # Salvar os mapeamentos para uso posterior na geração
    with open(NOME_ARQUIVO_MAPS, 'wb') as f:
        pickle.dump({'char_para_int': char_para_int, 'int_para_char': int_para_char}, f)

    # Criar sequências de treino
    step = 1
    entradas = []
    saidas = []
    for i in range(0, len(texto) - TAMANHO_SEQUENCIA, step):
        entradas.append(texto[i: i + TAMANHO_SEQUENCIA])
        saidas.append(texto[i + TAMANHO_SEQUENCIA])
    
    # Vetorização
    x = np.zeros((len(entradas), TAMANHO_SEQUENCIA, n_vocabulario), dtype=np.bool_)
    y = np.zeros((len(entradas), n_vocabulario), dtype=np.bool_)
    for i, sequencia in enumerate(entradas):
        for t, char in enumerate(sequencia):
            x[i, t, char_para_int[char]] = 1
        y[i, char_para_int[saidas[i]]] = 1

    # Construir o modelo
    modelo = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(256, input_shape=(TAMANHO_SEQUENCIA, n_vocabulario)),
        tf.keras.layers.Dense(n_vocabulario, activation='softmax')
    ])
    
    modelo.compile(optimizer=optimizer, loss='categorical_crossentropy')
    modelo.summary()

    # Treinar
    modelo.fit(x, y, batch_size=BATCH_SIZE, epochs=EPOCAS_TREINO, callbacks=cb)

    # Salvar o modelo treinado
    modelo.save(NOME_ARQUIVO_MODELO)
    print(f">>> Treinamento concluído! Modelo salvo como '{NOME_ARQUIVO_MODELO}'.")
    return modelo, char_para_int, int_para_char

In [3]:
# --- 3. FUNÇÃO DE GERAÇÃO DE TEXTO ---
def gerar_texto(modelo, char_para_int, int_para_char, prompt_inicial, num_caracteres_gerar):
    """
    Usa um modelo treinado para gerar texto a partir de um prompt.
    """
    n_vocabulario = len(char_para_int)
    sequencia_formatada = prompt_inicial.lower()
    
    # Garante que a semente tenha o tamanho correto
    if len(sequencia_formatada) < TAMANHO_SEQUENCIA:
        sequencia_formatada = " " * (TAMANHO_SEQUENCIA - len(sequencia_formatada)) + sequencia_formatada
    else:
        sequencia_formatada = sequencia_formatada[-TAMANHO_SEQUENCIA:]

    texto_gerado = ""
    sys.stdout.write("\n--- TEXTO GERADO ---\n")
    sys.stdout.write(prompt_inicial)

    for i in range(num_caracteres_gerar):
        x_pred = np.zeros((1, TAMANHO_SEQUENCIA, n_vocabulario), dtype=np.bool_)
        for t, char in enumerate(sequencia_formatada):
            # Ignora caracteres que não estavam no treino
            if char in char_para_int:
                x_pred[0, t, char_para_int[char]] = 1.

        preds = modelo.predict(x_pred, verbose=0)[0]
        # Amostragem com temperatura para tornar o texto menos repetitivo (opcional, mas melhor)
        # preds = np.asarray(preds).astype('float64')
        # preds = np.log(preds) / 0.5 # temperatura de 0.5
        # exp_preds = np.exp(preds)
        # preds = exp_preds / np.sum(exp_preds)
        # prox_indice = np.random.choice(len(preds), p=preds)
        #prox_indice = np.argmax(preds) # O jeito mais simples: pegar o mais provável
        prox_indice = sample_with_temperature(preds, temperature=0.2) 

        prox_char = int_para_char[prox_indice]
        
        texto_gerado += prox_char
        sequencia_formatada = sequencia_formatada[1:] + prox_char

        sys.stdout.write(prox_char)
        sys.stdout.flush()

    print("\n\n--- FIM DA GERAÇÃO ---")

In [6]:
def sample_with_temperature(preds, temperature=1.0):
    """
    Re-escala as previsões (logits) com a temperatura e retorna um índice amostrado.
    """
    # Converte para float64 para precisão
    preds = np.asarray(preds).astype('float64')
    
    # Evita divisão por zero e problemas com log
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)
    
    # Normaliza para que a soma das probabilidades seja 1
    preds = exp_preds / np.sum(exp_preds)
    
    # Amostra um índice com base na nova distribuição de probabilidade
    probas = np.random.multinomial(1, preds, 1)
    
    return np.argmax(probas)

In [7]:
# --- 4. SCRIPT PRINCIPAL ---
if __name__ == "__main__":
    if not os.path.exists(NOME_ARQUIVO_MODELO):
        modelo, char_para_int, int_para_char = treinar_modelo()
    else:
        print(f">>> Carregando modelo pré-treinado '{NOME_ARQUIVO_MODELO}'...")
        modelo = tf.keras.models.load_model(NOME_ARQUIVO_MODELO)
        with open(NOME_ARQUIVO_MAPS, 'rb') as f:
            mapeamentos = pickle.load(f)
            char_para_int = mapeamentos['char_para_int']
            int_para_char = mapeamentos['int_para_char']
        print(">>> Modelo carregado com sucesso!")

    # Loop de interação com o usuário
    while True:
        try:
            prompt = input("\nDigite o texto inicial (prompt) ou 'sair' para terminar: ")
            if prompt.lower() == 'sair':
                break
            
            num_chars_str = input("Quantos CARACTERES você deseja gerar? ")
            num_chars = int(num_chars_str)

            gerar_texto(modelo, char_para_int, int_para_char, prompt, num_chars)

        except ValueError:
            print("Erro: Por favor, digite um número válido de caracteres.")
        except KeyboardInterrupt:
            print("\nEncerrando o programa.")
            break
        except Exception as e:
            print(f"Ocorreu um erro inesperado: {e}")

>>> Carregando modelo pré-treinado 'modelo_char_rnn.keras'...
>>> Modelo carregado com sucesso!

--- TEXTO GERADO ---
boa tarde
            lxxxi     o desencio
             lxxxi     o retrato
             lxxxi     o retrato de
             lxxxvii   a proprio
             cxxxi     o pae de ser
            cxxxvii   

--- FIM DA GERAÇÃO ---

--- TEXTO GERADO ---
o trabalho de
claro da mão de dessei levara de dissimular a minha mãe era a
proprio para o pae de meu parecido e a minha mãe, e não sei a meus
perguntou-me a minha mãe era a alma a minha mãe e a minha mãe, e
a

--- FIM DA GERAÇÃO ---

--- TEXTO GERADO ---
a cada vez que
estava como a minha mãe estava de ser parece que estava a minha mãe
estava a minha parte de um annos, e o preto que eu não era a propria
a propria de capitú e a casa do parte de capitú e a minha mãe, e a
menina em casa, e a vida de capitú e o que era a minha mãe, e eu não
se perguntou a minha 

--- FIM DA GERAÇÃO ---
